In [1]:
import os
import glob
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col, to_date
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split


In [2]:
# Build a .py script that takes a snapshot date, trains a model and outputs artefact into storage.

## set up pyspark session

In [3]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/25 06:46:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## set up config

In [15]:
# set up config
model_train_date_str = "2024-03-01"
train_test_period_months = 12
oot_period_months = 2
train_test_ratio = 0.8

config = {}
config["model_train_date_str"] = model_train_date_str
config["train_test_period_months"] = train_test_period_months
config["oot_period_months"] =  oot_period_months
config["model_train_date"] =  datetime.strptime(model_train_date_str, "%Y-%m-%d")
config["oot_end_date"] =  config['model_train_date'] - timedelta(days = 1)
config["oot_start_date"] =  config['model_train_date'] - relativedelta(months = oot_period_months)
config["train_test_end_date"] =  config["oot_start_date"] - timedelta(days = 1)
config["train_test_start_date"] =  config["oot_start_date"] - relativedelta(months = train_test_period_months)
config["train_test_ratio"] = train_test_ratio 


pprint.pprint(config)

{'model_train_date': datetime.datetime(2024, 3, 1, 0, 0),
 'model_train_date_str': '2024-03-01',
 'oot_end_date': datetime.datetime(2024, 2, 29, 0, 0),
 'oot_period_months': 2,
 'oot_start_date': datetime.datetime(2024, 1, 1, 0, 0),
 'train_test_end_date': datetime.datetime(2023, 12, 31, 0, 0),
 'train_test_period_months': 12,
 'train_test_ratio': 0.8,
 'train_test_start_date': datetime.datetime(2023, 1, 1, 0, 0)}


## get label store

In [16]:
# connect to label store
folder_path = "datamart/gold/label_store/"
files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
label_store_sdf = spark.read.option("header", "true").parquet(*files_list)
print("row_count:",label_store_sdf.count())

label_store_sdf.show()

row_count: 9963
+--------------------+-----------+-----+----------+-------------+
|             loan_id|Customer_ID|label| label_def|snapshot_date|
+--------------------+-----------+-----+----------+-------------+
|CUS_0x1037_2023_0...| CUS_0x1037|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1069_2023_0...| CUS_0x1069|    0|30dpd_6mob|   2023-07-01|
|CUS_0x114a_2023_0...| CUS_0x114a|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1184_2023_0...| CUS_0x1184|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1297_2023_0...| CUS_0x1297|    1|30dpd_6mob|   2023-07-01|
|CUS_0x12fb_2023_0...| CUS_0x12fb|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1325_2023_0...| CUS_0x1325|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1341_2023_0...| CUS_0x1341|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1375_2023_0...| CUS_0x1375|    1|30dpd_6mob|   2023-07-01|
|CUS_0x13a8_2023_0...| CUS_0x13a8|    0|30dpd_6mob|   2023-07-01|
|CUS_0x13ef_2023_0...| CUS_0x13ef|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1440_2023_0...| CUS_0x1440|    0|30dpd_6mob|   2023-0

In [17]:
# extract label store
labels_sdf = label_store_sdf.filter((col("snapshot_date") >= config["train_test_start_date"]) & (col("snapshot_date") <= config["oot_end_date"]))

print("extracted labels_sdf", labels_sdf.count(), config["train_test_start_date"], config["oot_end_date"])

extracted labels_sdf 4037 2023-01-01 00:00:00 2024-02-29 00:00:00


## get features

In [66]:
gold_clks_directory = "datamart/gold/feature_store/eng/"

folder_path = gold_clks_directory
files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
feature_store_sdf_1 = spark.read.parquet(*files_list)
print("row_count:",feature_store_sdf_1.count())
feature_store_sdf_1.printSchema()
feature_store_sdf_1.show(5)

row_count: 206402
root
 |-- Customer_ID: string (nullable = true)
 |-- snapshot_date: string (nullable = true)
 |-- click_1m: integer (nullable = true)
 |-- click_2m: integer (nullable = true)
 |-- click_3m: integer (nullable = true)
 |-- click_4m: integer (nullable = true)
 |-- click_5m: integer (nullable = true)
 |-- click_6m: integer (nullable = true)

+-----------+-------------+--------+--------+--------+--------+--------+--------+
|Customer_ID|snapshot_date|click_1m|click_2m|click_3m|click_4m|click_5m|click_6m|
+-----------+-------------+--------+--------+--------+--------+--------+--------+
| CUS_0xc65a|   2024-03-01|     239|     236|      44|     309|       0|      35|
| CUS_0x5e1f|   2024-03-01|     103|      26|     182|     133|      27|      43|
| CUS_0x78d3|   2024-03-01|      69|     124|     140|       0|       0|     149|
| CUS_0x1844|   2024-03-01|     142|     232|      25|     145|       0|      97|
| CUS_0x7f07|   2024-03-01|      53|       0|     256|     158|     

In [67]:
gold_fin_directory = "datamart/gold/feature_store/cust_fin_risk/"

folder_path = gold_fin_directory
files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
feature_store_sdf_2 = spark.read.parquet(*files_list)
print("row_count:",feature_store_sdf_2.count())
feature_store_sdf_2.printSchema()
feature_store_sdf_2.show(5)

row_count: 11974
root
 |-- Customer_ID: string (nullable = true)
 |-- snapshot_date: date (nullable = true)
 |-- Credit_History_Age: integer (nullable = true)
 |-- Num_Fin_Pdts: integer (nullable = true)
 |-- EMI_to_Salary: double (nullable = true)
 |-- Debt_to_Salary: double (nullable = true)
 |-- Repayment_Ability: double (nullable = true)
 |-- Loans_per_Credit_Item: double (nullable = true)
 |-- Loan_Extent: integer (nullable = true)
 |-- Outstanding_Debt: double (nullable = true)
 |-- Interest_Rate: integer (nullable = true)
 |-- Delay_from_due_date: integer (nullable = true)
 |-- Changed_Credit_Limit: double (nullable = true)

+-----------+-------------+------------------+------------+--------------------+-------------------+-----------------+---------------------+-----------+----------------+-------------+-------------------+--------------------+
|Customer_ID|snapshot_date|Credit_History_Age|Num_Fin_Pdts|       EMI_to_Salary|     Debt_to_Salary|Repayment_Ability|Loans_per_Credit_

In [68]:
feature_store_sdf_1 = feature_store_sdf_1.withColumn("snapshot_date", to_date(col("snapshot_date")))
feature_store_sdf_2 = feature_store_sdf_2.withColumn("snapshot_date", to_date(col("snapshot_date")))
feature_store_sdf_1.printSchema()
feature_store_sdf_2.printSchema()

root
 |-- Customer_ID: string (nullable = true)
 |-- snapshot_date: date (nullable = true)
 |-- click_1m: integer (nullable = true)
 |-- click_2m: integer (nullable = true)
 |-- click_3m: integer (nullable = true)
 |-- click_4m: integer (nullable = true)
 |-- click_5m: integer (nullable = true)
 |-- click_6m: integer (nullable = true)

root
 |-- Customer_ID: string (nullable = true)
 |-- snapshot_date: date (nullable = true)
 |-- Credit_History_Age: integer (nullable = true)
 |-- Num_Fin_Pdts: integer (nullable = true)
 |-- EMI_to_Salary: double (nullable = true)
 |-- Debt_to_Salary: double (nullable = true)
 |-- Repayment_Ability: double (nullable = true)
 |-- Loans_per_Credit_Item: double (nullable = true)
 |-- Loan_Extent: integer (nullable = true)
 |-- Outstanding_Debt: double (nullable = true)
 |-- Interest_Rate: integer (nullable = true)
 |-- Delay_from_due_date: integer (nullable = true)
 |-- Changed_Credit_Limit: double (nullable = true)



In [69]:
# extract feature store
features_sdf_1 = feature_store_sdf_1.filter((col("snapshot_date") >= config["train_test_start_date"]) & (col("snapshot_date") <= config["oot_end_date"]))
features_sdf_2 = feature_store_sdf_2
features_sdf_2 = features_sdf_2.drop('snapshot_date')

print("extracted features_sdf_1", features_sdf_1.count(), config["train_test_start_date"], config["oot_end_date"])
print("extracted features_sdf_2", features_sdf_2.count(), config["train_test_start_date"], config["oot_end_date"])

extracted features_sdf_1 125636 2023-07-01 00:00:00 2024-08-31 00:00:00
extracted features_sdf_2 11974 2023-07-01 00:00:00 2024-08-31 00:00:00


In [70]:
# join two feature tables
features_sdf = features_sdf_1.join(features_sdf_2, on=["Customer_ID"], how="left")

print("extracted features_sdf", features_sdf.count(), config["train_test_start_date"], config["oot_end_date"])
features_sdf.show(5)

extracted features_sdf 125636 2023-07-01 00:00:00 2024-08-31 00:00:00
+-----------+-------------+--------+--------+--------+--------+--------+--------+------------------+------------+--------------------+------------------+------------------+---------------------+-----------+----------------+-------------+-------------------+--------------------+
|Customer_ID|snapshot_date|click_1m|click_2m|click_3m|click_4m|click_5m|click_6m|Credit_History_Age|Num_Fin_Pdts|       EMI_to_Salary|    Debt_to_Salary| Repayment_Ability|Loans_per_Credit_Item|Loan_Extent|Outstanding_Debt|Interest_Rate|Delay_from_due_date|Changed_Credit_Limit|
+-----------+-------------+--------+--------+--------+--------+--------+--------+------------------+------------+--------------------+------------------+------------------+---------------------+-----------+----------------+-------------+-------------------+--------------------+
| CUS_0xc65a|   2024-03-01|     239|     236|      44|     309|       0|      35|            

In [71]:
features_pdf = features_sdf.toPandas()
columns_to_exclude = ['Customer_ID', 'snapshot_date']
columns_to_rename = [col for col in features_pdf.columns if col not in columns_to_exclude]
rename_dict = {col: 'feature_' + col for col in columns_to_rename}
features_pdf.rename(columns=rename_dict, inplace=True)
features_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125636 entries, 0 to 125635
Data columns (total 19 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Customer_ID                    125636 non-null  object 
 1   snapshot_date                  125636 non-null  object 
 2   feature_click_1m               125636 non-null  int32  
 3   feature_click_2m               125636 non-null  int32  
 4   feature_click_3m               125636 non-null  int32  
 5   feature_click_4m               125636 non-null  int32  
 6   feature_click_5m               125636 non-null  int32  
 7   feature_click_6m               125636 non-null  int32  
 8   feature_Credit_History_Age     125636 non-null  int32  
 9   feature_Num_Fin_Pdts           125636 non-null  int32  
 10  feature_EMI_to_Salary          125636 non-null  float64
 11  feature_Debt_to_Salary         125636 non-null  float64
 12  feature_Repayment_Ability     

In [72]:
features_sdf = spark.createDataFrame(features_pdf)
features_sdf.show(5)

+-----------+-------------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------------+--------------------+---------------------+----------------------+-------------------------+-----------------------------+-------------------+------------------------+---------------------+---------------------------+----------------------------+
|Customer_ID|snapshot_date|feature_click_1m|feature_click_2m|feature_click_3m|feature_click_4m|feature_click_5m|feature_click_6m|feature_Credit_History_Age|feature_Num_Fin_Pdts|feature_EMI_to_Salary|feature_Debt_to_Salary|feature_Repayment_Ability|feature_Loans_per_Credit_Item|feature_Loan_Extent|feature_Outstanding_Debt|feature_Interest_Rate|feature_Delay_from_due_date|feature_Changed_Credit_Limit|
+-----------+-------------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------------+--------------------+-----------------

## prepare data for modeling

In [73]:
# prepare data for modeling
data_pdf = labels_sdf.join(features_sdf, on=["Customer_ID", "snapshot_date"], how="left").toPandas()
data_pdf

,Customer_ID,snapshot_date,loan_id,label,label_def,feature_click_1m,feature_click_2m,feature_click_3m,feature_click_4m,feature_click_5m,...,feature_Num_Fin_Pdts,feature_EMI_to_Salary,feature_Debt_to_Salary,feature_Repayment_Ability,feature_Loans_per_Credit_Item,feature_Loan_Extent,feature_Outstanding_Debt,feature_Interest_Rate,feature_Delay_from_due_date,feature_Changed_Credit_Limit
0,CUS_0x10eb,2023-09-01,CUS_0x10eb_2023_03_01,0,30dpd_6mob,63,0,167,216,66,...,13,0.012509,0.280304,2385.432,0.166667,16,677.40,10,8,10.34
1,CUS_0x112f,2024-04-01,CUS_0x112f_2023_10_01,1,30dpd_6mob,136,55,160,89,124,...,6,0.010065,0.176691,5989.298,0.166667,1,1069.19,5,1,3.44
2,CUS_0x117d,2024-03-01,CUS_0x117d_2023_09_01,0,30dpd_6mob,28,317,0,89,146,...,9,0.022406,0.228633,5729.917,0.666667,56,1340.31,9,14,3.61
3,CUS_0x1192,2023-11-01,CUS_0x1192_2023_05_01,0,30dpd_6mob,0,0,85,80,69,...,12,0.010718,0.839055,1502.657,0.181818,32,1275.32,1,16,7.67
4,CUS_0x12dd,2023-09-01,CUS_0x12dd_2023_03_01,0,30dpd_6mob,94,131,34,183,0,...,8,0.030106,0.254875,2464.653,0.800000,40,647.94,11,10,6.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6956,CUS_0xfa4,2024-08-01,CUS_0xfa4_2024_02_01,1,30dpd_6mob,254,62,263,47,63,...,24,0.032855,1.190460,1174.983,0.250000,305,1447.52,34,61,10.98
6957,CUS_0xfaa,2024-01-01,CUS_0xfaa_2023_07_01,0,30dpd_6mob,149,15,257,143,222,...,23,0.023844,2.646690,1721.735,0.263158,110,4670.92,31,22,18.16
6958,CUS_0xfb4,2024-06-01,CUS_0xfb4_2023_12_01,0,30dpd_6mob,0,234,27,227,118,...,17,0.040744,1.564651,893.244,0.285714,48,1458.61,12,12,10.45
6959,CUS_0xfc9,2023-07-01,CUS_0xfc9_2023_01_01,1,30dpd_6mob,28,0,30,182,0,...,17,0.045535,1.116812,1361.850,0.636364,14,1594.66,31,2,16.86


In [74]:
# split data into train - test - oot
oot_pdf = data_pdf[(data_pdf['snapshot_date'] >= config["oot_start_date"].date()) & (data_pdf['snapshot_date'] <= config["oot_end_date"].date())]
train_test_pdf = data_pdf[(data_pdf['snapshot_date'] >= config["train_test_start_date"].date()) & (data_pdf['snapshot_date'] <= config["train_test_end_date"].date())]

feature_cols = [fe_col for fe_col in data_pdf.columns if fe_col.startswith('feature_')]

X_oot = oot_pdf[feature_cols]
y_oot = oot_pdf["label"]
X_train, X_test, y_train, y_test = train_test_split(
    train_test_pdf[feature_cols], train_test_pdf["label"], 
    test_size= 1 - config["train_test_ratio"],
    random_state=88,     # Ensures reproducibility
    shuffle=True,        # Shuffle the data before splitting
    stratify=train_test_pdf["label"]           # Stratify based on the label column
)


print('X_train', X_train.shape[0])
print('X_test', X_test.shape[0])
print('X_oot', X_oot.shape[0])
print('y_train', y_train.shape[0], round(y_train.mean(),2))
print('y_test', y_test.shape[0], round(y_test.mean(),2))
print('y_oot', y_oot.shape[0], round(y_oot.mean(),2))

X_train

X_train 4766
X_test 1192
X_oot 1003
y_train 4766 0.28
y_test 1192 0.28
y_oot 1003 0.29


,feature_click_1m,feature_click_2m,feature_click_3m,feature_click_4m,feature_click_5m,feature_click_6m,feature_Credit_History_Age,feature_Num_Fin_Pdts,feature_EMI_to_Salary,feature_Debt_to_Salary,feature_Repayment_Ability,feature_Loans_per_Credit_Item,feature_Loan_Extent,feature_Outstanding_Debt,feature_Interest_Rate,feature_Delay_from_due_date,feature_Changed_Credit_Limit
171,67,98,91,3,82,116,316,9,0.011823,0.090926,10374.739,0.250000,40,954.71,20,20,6.82
4188,188,11,148,304,5,83,215,19,0.010434,0.894752,1714.686,0.111111,68,1551.30,20,34,14.09
3399,20,194,305,296,0,82,234,8,0.000000,0.319262,1769.398,0.000000,0,565.22,34,8,8.61
523,65,170,180,169,71,146,348,9,0.009902,0.004169,10138.545,0.250000,20,42.69,1,10,8.84
4255,0,44,162,44,309,102,185,22,0.052180,0.849828,1455.926,0.352941,198,1306.30,28,33,1.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025,90,6,0,246,207,96,207,6,0.000000,0.184354,5497.448,0.000000,0,1013.66,1,5,11.47
4288,154,61,160,98,0,0,85,19,0.045284,0.323540,4508.151,0.333333,125,1528.09,33,25,18.38
2152,77,168,23,214,0,0,96,12,0.016332,0.127992,4112.972,0.300000,75,535.30,14,25,11.28
6295,205,0,80,254,0,115,11,23,0.055329,8.068782,461.597,0.263158,150,3951.21,15,30,8.18


## preprocess data

In [76]:
# set up standard scalar preprocessing
scaler = StandardScaler()

transformer_stdscaler = scaler.fit(X_train) # Q which should we use? train? test? oot? all?

# transform data
X_train_processed = transformer_stdscaler.transform(X_train)
X_test_processed = transformer_stdscaler.transform(X_test)
X_oot_processed = transformer_stdscaler.transform(X_oot)

print('X_train_processed', X_train_processed.shape[0])
print('X_test_processed', X_test_processed.shape[0])
print('X_oot_processed', X_oot_processed.shape[0])

pd.DataFrame(X_train_processed)

X_train_processed 4766
X_test_processed 1192
X_oot_processed 1003


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,-0.490364,-0.128866,-0.223693,-1.209670,-0.345725,0.057214,0.915602,-1.007557,-0.119947,-0.560585,0.856356,-0.297246,-0.486202,-0.407545,0.555992,-0.060286,-0.531908
1,0.899563,-1.114448,0.430651,2.237133,-1.226011,-0.317409,-0.100691,0.737996,-0.120258,0.141751,-0.120607,-0.881192,-0.241290,0.110008,0.555992,0.894052,0.566368
2,-1.030254,0.958672,2.232967,2.145523,-1.283173,-0.328762,0.090493,-1.182112,-0.122599,-0.361078,-0.114435,-1.348349,-0.836077,-0.745435,2.076837,-0.878290,-0.261493
3,-0.513338,0.686788,0.798002,0.691225,-0.471480,0.397780,1.237596,-1.007557,-0.120378,-0.636388,0.829711,-0.297246,-0.661139,-1.198740,-1.508013,-0.741956,-0.226747
4,-1.259994,-0.740607,0.591367,-0.740172,2.249405,-0.101717,-0.402560,1.261662,-0.110895,0.102499,-0.149798,0.135561,0.895803,-0.102534,1.425046,0.825885,-1.296320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4761,-0.226163,-1.171091,-1.268348,1.572965,1.083312,-0.169831,-0.181189,-1.531222,-0.122599,-0.478953,0.306136,-1.348349,-0.836077,-0.356405,-1.508013,-1.082791,0.170566
4762,0.509005,-0.548022,0.568408,-0.121809,-1.283173,-1.259643,-1.408791,0.737996,-0.112442,-0.357340,0.194531,0.053122,0.257282,0.089873,1.968205,0.280549,1.214457
4763,-0.375494,0.664131,-1.004314,1.206527,-1.283173,-1.259643,-1.298105,-0.483891,-0.118936,-0.528198,0.149950,-0.087025,-0.180061,-0.771391,-0.095799,0.280549,0.141863
4764,1.094842,-1.239062,-0.349970,1.664574,-1.283173,0.045861,-2.153401,1.436217,-0.110189,6.409994,-0.261971,-0.241925,0.475954,2.191977,0.012833,0.621384,-0.326453


## train model

In [77]:
# Define the XGBoost classifier
xgb_clf = xgb.XGBClassifier(eval_metric='logloss', random_state=88)

# Define the hyperparameter space to search
param_dist = {
    'n_estimators': [25, 50],
    'max_depth': [2, 3],  # lower max_depth to simplify the model
    'learning_rate': [0.01, 0.1],
    'subsample': [0.6, 0.8],
    'colsample_bytree': [0.6, 0.8],
    'gamma': [0, 0.1],
    'min_child_weight': [1, 3, 5],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [1, 1.5, 2]
}

# Create a scorer based on AUC score
auc_scorer = make_scorer(roc_auc_score)

# Set up the random search with cross-validation
random_search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_dist,
    scoring=auc_scorer,
    n_iter=100,  # Number of iterations for random search
    cv=3,       # Number of folds in cross-validation
    verbose=1,
    random_state=42,
    n_jobs=-1   # Use all available cores
)

# Perform the random search
random_search.fit(X_train_processed, y_train)

# Output the best parameters and best score
print("Best parameters found: ", random_search.best_params_)
print("Best AUC score: ", random_search.best_score_)

# Evaluate the model on the train set
best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_train_processed)[:, 1]
train_auc_score = roc_auc_score(y_train, y_pred_proba)
print("Train AUC score: ", train_auc_score)

# Evaluate the model on the test set
best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_test_processed)[:, 1]
test_auc_score = roc_auc_score(y_test, y_pred_proba)
print("Test AUC score: ", test_auc_score)

# Evaluate the model on the oot set
best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_oot_processed)[:, 1]
oot_auc_score = roc_auc_score(y_oot, y_pred_proba)
print("OOT AUC score: ", oot_auc_score)

print("TRAIN GINI score: ", round(2*train_auc_score-1,3))
print("Test GINI score: ", round(2*test_auc_score-1,3))
print("OOT GINI score: ", round(2*oot_auc_score-1,3))

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best parameters found:  {'subsample': 0.8, 'reg_lambda': 1, 'reg_alpha': 1, 'n_estimators': 50, 'min_child_weight': 3, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 0.8}
Best AUC score:  0.7025970696758576
Train AUC score:  0.8369417248992468
Test AUC score:  0.768132993214986
OOT AUC score:  0.7901849492258388
TRAIN GINI score:  0.674
Test GINI score:  0.536
OOT GINI score:  0.58


## prepare model artefact to save

In [78]:
model_artefact = {}

model_artefact['model'] = best_model
model_artefact['model_version'] = "credit_model_"+config["model_train_date_str"].replace('-','_')
model_artefact['preprocessing_transformers'] = {}
model_artefact['preprocessing_transformers']['stdscaler'] = transformer_stdscaler
model_artefact['data_dates'] = config
model_artefact['data_stats'] = {}
model_artefact['data_stats']['X_train'] = X_train.shape[0]
model_artefact['data_stats']['X_test'] = X_test.shape[0]
model_artefact['data_stats']['X_oot'] = X_oot.shape[0]
model_artefact['data_stats']['y_train'] = round(y_train.mean(),2)
model_artefact['data_stats']['y_test'] = round(y_test.mean(),2)
model_artefact['data_stats']['y_oot'] = round(y_oot.mean(),2)
model_artefact['results'] = {}
model_artefact['results']['auc_train'] = train_auc_score
model_artefact['results']['auc_test'] = test_auc_score
model_artefact['results']['auc_oot'] = oot_auc_score
model_artefact['results']['gini_train'] = round(2*train_auc_score-1,3)
model_artefact['results']['gini_test'] = round(2*test_auc_score-1,3)
model_artefact['results']['gini_oot'] = round(2*oot_auc_score-1,3)
model_artefact['hp_params'] = random_search.best_params_


pprint.pprint(model_artefact)

{'data_dates': {'model_train_date': datetime.datetime(2024, 9, 1, 0, 0),
                'model_train_date_str': '2024-09-01',
                'oot_end_date': datetime.datetime(2024, 8, 31, 0, 0),
                'oot_period_months': 2,
                'oot_start_date': datetime.datetime(2024, 7, 1, 0, 0),
                'train_test_end_date': datetime.datetime(2024, 6, 30, 0, 0),
                'train_test_period_months': 12,
                'train_test_ratio': 0.8,
                'train_test_start_date': datetime.datetime(2023, 7, 1, 0, 0)},
 'data_stats': {'X_oot': 1003,
                'X_test': 1192,
                'X_train': 4766,
                'y_oot': np.float64(0.29),
                'y_test': np.float64(0.28),
                'y_train': np.float64(0.28)},
 'hp_params': {'colsample_bytree': 0.8,
               'gamma': 0,
               'learning_rate': 0.1,
               'max_depth': 3,
               'min_child_weight': 3,
               'n_estimators': 50,
          

## save artefact to model bank

In [79]:
# create model_bank dir
model_bank_directory = "model_bank/"

if not os.path.exists(model_bank_directory):
    os.makedirs(model_bank_directory)

In [80]:
# Full path to the file
file_path = os.path.join(model_bank_directory, model_artefact['model_version'] + '.pkl')

# Write the model to a pickle file
with open(file_path, 'wb') as file:
    pickle.dump(model_artefact, file)

print(f"Model saved to {file_path}")


Model saved to model_bank/credit_model_2024_09_01.pkl


## test load pickle and make model inference

In [81]:
# Load the model from the pickle file
with open(file_path, 'rb') as file:
    loaded_model_artefact = pickle.load(file)

y_pred_proba = loaded_model_artefact['model'].predict_proba(X_oot_processed)[:, 1]
oot_auc_score = roc_auc_score(y_oot, y_pred_proba)
print("OOT AUC score: ", oot_auc_score)

print("Model loaded successfully!")

OOT AUC score:  0.7901849492258388
Model loaded successfully!
